<a href="https://colab.research.google.com/github/TurkuNLP/Turku-paraphrase-models/blob/main/para_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains a simple application of the FinParaphrase model. The FinParaphrase model is a Finnish paraphrase classification model based on FinBERT. The model takes paraphrase pairs as input, and the output includes the base label as well as the additional flags as described in the FinParaphrase annotation manual (link forthcoming).

First, install all necessary dependencies and download the models and the associated code.

In [1]:
!pip install transformers
!pip install pytorch-lightning
!git clone https://github.com/TurkuNLP/paraphrase-classification para
!wget http://dl.turkunlp.org/finbert/torch-transformers/bert-large-finnish-cased-rc2/bert-large-finnish-cased-rc2.tar.gz
!tar zxf bert-large-finnish-cased-rc2.tar.gz

     |████████████████████████████████| 2.6 MB 6.6 MB/s 
     |████████████████████████████████| 636 kB 63.0 MB/s 
     |████████████████████████████████| 895 kB 65.9 MB/s 
     |████████████████████████████████| 3.3 MB 58.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 916 kB 7.0 MB/s 
     |████████████████████████████████| 272 kB 73.3 MB/s 
     |████████████████████████████████| 118 kB 65.7 MB/s 
     |████████████████████████████████| 829 kB 58.4 MB/s 
     |████████████████████████████████| 1.3 MB 59.8 MB/s 
     |████████████████████████████████| 142 kB 68.0 MB/s 
     |████████████████████████████████| 294 kB 71.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=68b8bb33936d0c93e614219960c6da7f8c453915d088c03767d36e5a6cee4c03
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17

Then set up the model and write some helper functions.

In [2]:
import torch
import transformers
from para.notebook import para_model

bert_model = "bert-large-finnish-cased-rc2"

model = para_model.ParaMultiOutputAvgModel.load_from_checkpoint(checkpoint_path="http://dl.turkunlp.org/turku-paraphrase/model.ckpt", bert_model=bert_model)

model.eval()
model.cuda()

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

def compute_masks(mask):
  one_idx = [i for i, b in enumerate(mask) if b]
  zeros = torch.zeros(len(mask), dtype=torch.long)
  cls_mask, sep1_mask, sep2_mask = [torch.scatter(zeros, 0, torch.tensor(i), torch.tensor(1)) for i in one_idx]
  left_idx = torch.tensor(range(one_idx[0]+1, one_idx[1]))
  left_mask = torch.scatter(zeros, 0, left_idx, torch.ones(len(left_idx), dtype=torch.long))
  right_idx = torch.tensor(range(one_idx[1]+1, one_idx[2]))
  right_mask = torch.scatter(zeros, 0, right_idx, torch.ones(len(right_idx), dtype=torch.long))
  return {'cls_mask': cls_mask, 'sep1_mask': sep1_mask, 'sep2_mask': sep2_mask, 'left_mask': left_mask, 'right_mask': right_mask}

def encode_batch(tokenizer, txt1, txt2):
  encoded = tokenizer(txt1, txt2, return_special_tokens_mask=True, return_length=True, max_length=512, truncation=True, padding='longest', return_tensors='pt')
  masks = [compute_masks(v) for v in encoded.special_tokens_mask & encoded.attention_mask]
  mask_dict = {k: torch.stack([d[k] for d in masks]) for k in masks[0].keys()}
  return {"input_ids":encoded.input_ids, "token_type_ids":encoded.token_type_ids, "attention_mask":encoded.attention_mask, "length":encoded.length, **mask_dict}

Downloading: "http://dl.turkunlp.org/turku-paraphrase/model.ckpt" to /root/.cache/torch/hub/checkpoints/model.ckpt


  0%|          | 0.00/1.32G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-finnish-cased-rc2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Accuracy`

Finally, run paraphrase candidates through the model.

In [3]:
with torch.no_grad():
  batch = [("Tämä kakku on herkullista.", "Onpa hyvää kakkua."), ("Tämäpä vasta oli hienoa.", "Se oli ihan sairaan siistiä."), ("Turun yliopisto on ensimmäinen täysin suomenkielinen yliopisto, joka perustettiin vuonna 1920.", "Maailman nopeimpiin kuuluva supertietokone, LUMI, aloittaa toimintansa CSC:n datakeskuksessa Kajaanissa vuonna 2021.")]
  r = model({k: v.to(device='cuda') for k, v in encode_batch(tokenizer, *[l for l in zip(*batch)]).items()})
  indices = {k: torch.argmax(v, 1) for k, v in r.items()}
  flag_i2lab = {k: {vd: kd for kd, vd in v.items()} for k, v in model.flag_lab2i.items()}
  label_dict = {k: [flag_i2lab[k][x.item()] for x in v] for k, v in indices.items()}
  label = [b+d+i+s if b is '4' else b for b,d,i,s in zip(label_dict['base'], [d if d != 'None' else '' for d in label_dict['direction']], ['i' if i else '' for i in label_dict['has_i']], ['s' if s else '' for s in label_dict['has_s']])]
  print(label)

['4>', '4is', '2']
